In [1]:
# from moviepy.editor import VideoFileClip
from pathlib import Path
import os
import numpy as np
import pandas as pd
import random

import tensorflow.compat.v1 as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import keras_cv
import keras

from tqdm import tqdm
warnings.filterwarnings("ignore", category=UserWarning) #used to supress the tf version warning. 

mids_dir = Path("D:\\MIDS-W207")
data = mids_dir/"MIDS-W207-Spring24-Soccer-Detection-Data"
project = mids_dir/"MIDS-W207-Spring24-Soccer-Detection"
analysis = project/"analysis"

# Author: Timothy Majidzadeh
# Date Created: March 4, 2024
# Date Updated: March 5, 2024
# Description: Develop the baseline model.
# Notes: [v1] Created program.
# Inputs: Frame-by-frame image data & labels.
# Outputs: Object detection model?

NameError: name 'warnings' is not defined

In [ ]:
first_img = load_img(data/"base/top_view/D_20220220_1_0000_0030/D_20220220_1_0000_0030_1.png")
first_img

In [ ]:
labels = pd.read_pickle(data/"base/top_view_labels/top_view_labels.pkl")

In [ ]:
target_width, target_height = 1280, 736
train_pct, val_pct = 0.6, 0.4
batch_size = 8
learning_rate = 0.001
epoch = 5
global_clipnorm = 10
class_ids = [
    "ball"
]
class_mapping = dict(zip(range(len(class_ids)), class_ids))

In [ ]:
def scale_labels(input_df, target_height, target_width):
    resized_labels = input_df.copy()
    resized_labels.loc[
        : , (slice(None), slice(None), ['bb_width', 'bb_left']
    )] = round(resized_labels.loc[: , (slice(None), slice(None), ['bb_width', 'bb_left'])] * target_width / 3840)
    resized_labels.loc[
        : , (slice(None), slice(None), ['bb_height', 'bb_top'])
    ] = round(resized_labels.loc[: , (slice(None), slice(None), ['bb_height', 'bb_top'])] * target_height / 2160)
    return resized_labels

In [ ]:
# resized_labels = scale_labels(labels, target_height=target_height, target_width=target_width)
resized_labels = labels.copy()
resized_filtered_labels = resized_labels[(resized_labels['frame_saved'] == True)]
sampled_labels = resized_filtered_labels.sample(frac = 0.005, random_state=507401).reset_index()
sampled_labels

In [ ]:
def get_bbs(input_df):
    classes = tf.constant([[0] for i in input_df.index])
    boxes = tf.constant([
        [
            [
                input_df['ball']['ball']['bb_left'][i], input_df['ball']['ball']['bb_top'][i],
                input_df['ball']['ball']['bb_width'][i], input_df['ball']['ball']['bb_height'][i]
            ]
        ]
    for i in input_df.index])
    return classes, boxes

In [ ]:
classes, boxes = get_bbs(sampled_labels)
paths = sampled_labels["frame_imgpath"]
# images = [img_to_array(load_img(path, target_size=(target_height, target_width))) for path in paths]
tf_data = tf.data.Dataset.from_tensor_slices((
    tf.constant(paths),
    classes,
    boxes
))
tf_data

In [ ]:
print(classes)
print(boxes)

In [ ]:
# Split the list of sampled images into train, val & test.
train_index = int(round(sampled_labels.shape[0] * train_pct))
train = tf_data.take(train_index)
val = tf_data.skip(train_index)

In [ ]:
def load_image(image_path):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_png(image, channels=3)
    return image

def load_dataset(image_path, classes, bbox):
    # Read Image
    image = load_image(image_path)
    bounding_boxes = {
        "classes": tf.cast(classes, dtype=tf.float32),
        "boxes": bbox
    }
    print(bounding_boxes)
    return {"images": tf.cast(image, tf.float32), "bounding_boxes": bounding_boxes}

In [ ]:
augmenter = keras.Sequential(
    layers=[
        keras_cv.layers.Resizing(
            height=target_height,
            width=target_width,
            interpolation="bilinear",
            crop_to_aspect_ratio=False,
            pad_to_aspect_ratio=True,
            bounding_box_format="xywh"
        ),
        keras_cv.layers.RandomFlip(mode="horizontal", bounding_box_format="xywh")
    ]
)

In [ ]:
resizing = keras_cv.layers.Resizing(
    height=target_height,
    width=target_width,
    interpolation="bilinear",
    crop_to_aspect_ratio=False,
    pad_to_aspect_ratio=True,
    bounding_box_format="xywh"
)

In [ ]:
train, val = train.map(load_dataset, num_parallel_calls=tf.data.AUTOTUNE), val.map(load_dataset, num_parallel_calls=tf.data.AUTOTUNE)

train, val = train.shuffle(batch_size * 4), val.shuffle(batch_size * 4)
print(train)
train, val = train.ragged_batch(batch_size, drop_remainder=True), val.ragged_batch(batch_size, drop_remainder=True)
print(train)
train = train.map(augmenter, num_parallel_calls=tf.data.AUTOTUNE)
val = val.map(resizing, num_parallel_calls=tf.data.AUTOTUNE)

In [ ]:
def dict_to_tuple(inputs):
    return inputs["images"], inputs["bounding_boxes"]

# train_as_tuple, val_as_tuple = train.map(dict_to_tuple), val.map(dict_to_tuple)
# train, val = train.prefetch(tf.data.AUTOTUNE), val.prefetch(tf.data.AUTOTUNE)

In [ ]:
# Make the model!
optimizer = keras.optimizers.Adam(
    learning_rate=learning_rate, global_clipnorm=global_clipnorm
)
backbone = keras_cv.models.YOLOV8Backbone.from_preset(
    "yolo_v8_s_backbone"
)
model = keras_cv.models.YOLOV8Detector(
    backbone=backbone,
    num_classes=len(class_mapping),
    bounding_box_format="xywh",
    fpn_depth=1
)

In [ ]:
coco_metrics_callback = keras_cv.callbacks.PyCOCOCallback(
    val, bounding_box_format="xywh"
)
model.compile(
    classification_loss="binary_crossentropy",
    box_loss="ciou",
    optimizer=optimizer
)
model.fit(
    train,
    validation_data=val,
    epochs=epoch
)

In [ ]:
model.summary()

In [ ]:
model.save_weights(project/"analysis/Yolov8/YOLOv8_Baseline_v2")

In [ ]:
val

In [ ]:
model.evaluate(val)